## Notes: 
line of thinking
documented motivation
understand what er are doing


proprocess 3 subjects and provide/document the cleaning times / bad channels, document the bad components

- Preprocessing Filtering, re-referencening, ICA
- Data cleaning: Time, channel and subjects
- ERP peak analysis Extract the study-relevant ERP peak subjectwise (e.g. one value per subject) and statistically test them. RQ: On which ERP-peaks do we find major difference between the conditions?
- Time Frequency analysis Calculate an induced time-frequency analysis of the main experimental contrast RQ: What oscillations underley our effect of interest?
- either:	- Mass Univariate Use a multiple regression of the main experimental contrast, controlling for reaction time (you need to calculate RT yourself). RQ: When/Where do we find differences between our conditions? Is there an influence of reaction time?
		OR
		- Decoding analysis Decode the main contrast of the experiment across time RQ: When is information about the conditions in our data available?

## Dataset Summary and Exploration
P300: A visual oddball experiment with a prolonged effect at 300ms.

In [1]:
"""
import libraries
"""
import mne
import ccs_eeg_utils
from mne.channels import compute_native_head_t, read_custom_montage
from mne_bids import BIDSPath, read_raw_bids
from mne.datasets import fetch_fsaverage
from mne.viz import plot_alignment
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib
matplotlib.use("Agg")
#%matplotlib inline
import os  # make dir
import os.path as op
from config import *
import numpy as np
from pyvistaqt import BackgroundPlotter
import seaborn as sns
import pandas as pd


In [2]:
def downsample():
    print('before resampling:', raw.info)
    raw.resample(
        sfreq, npad="auto"
    )  # set sampling frequency to 256 Hz as in Kappenman et al.
    print('after resampling:', raw.info)

In [3]:
def rereference():
    # rerefernecing to P9 and P10 becuase Kappenman et al. says
    # "we find that P9 and P10 provide cleaner signals than the traditional mastoid sites"
    raw.plot(n_channels=len(raw.ch_names), duration = 5, start = 18)
    ccs_eeg_utils.save_plot("000beforeRereferencing", bids_path)
    raw.set_eeg_reference(ref_channels=ref_channels, verbose=None)
    raw.plot(n_channels=len(raw.ch_names), duration = 5, start = 18)
    ccs_eeg_utils.save_plot("000rereferencedToP9P10", bids_path)

In [4]:
def apply_montage():
    # Set EEG sensor configuration and head digitization
    raw.set_montage("standard_1020", match_case=False)
    raw.plot_sensors(ch_type='eeg',show_names=True)
    ccs_eeg_utils.save_plot("00montage", bids_path)    
    print(raw.info)


In [5]:
def band_pass_filtering(subject_id):
    """
    applys the firwin banpassfilter
    """
    print("filtering ...")

    # plot raw data before band-pass filtering
    raw.plot_psd(
        area_mode="range",
        tmax=10.0,
        average=False,
        xscale="linear",
    )
    ccs_eeg_utils.save_plot("01freq_before_filtering", bids_path)

    # apply filter
    raw_f = raw.copy().filter(l_freq, h_freq, fir_design=fir_design)
    raw_f.plot_psd(area_mode="range", tmax=10.0, average=False, xscale="log")
    ccs_eeg_utils.save_plot("02freq_after_filtering", bids_path)
    
    return raw_f

In [6]:
def plot_each_channel(subject_id):
    """
    plot data
    """
    print("plotting each channel ...")

    # create dir to save plots if not existent
    if not os.path.exists(str("./../results/" + str(bids_path)[14:-37]) + str("03whole_overlay/")):
        os.makedirs(
            str("./../results/" + str(bids_path)[14:-37]) + str("03whole_overlay/")
        )

    # Extract a single channel and plot the whole timeseries.
    for channel_number in range(1, len(raw.ch_names)):  # 1

        # show filtered -> low frequencies
        fig = plt.figure()
        a1 = fig.add_axes([0, 0, 1, 1])
        a1.plot((raw[channel_number - 1][0].T))
        a1.plot((raw_f[channel_number - 1][0].T))
        a1.legend(["raw", "band-pass filtered"])
        plt.title("Subject: " + str(subject_id) + ", channel: " + str(channel_number))
        plt.ylabel("potential [µV]")
        plt.xlabel("sample")
        # a1.set_xlim(0,25)
        # a1.set_ylim(-0.00003,0.00003)
        plt.savefig(
            str("./../results/" + str(bids_path)[14:-37])
            + str("03whole_overlay/channel" + str(channel_number) + ".png"),
            bbox_inches="tight",
        )
        # plt.show()
        plt.close()  # don't display figure

    # show raw -> high frequnecies
    channel_number = 2
    fig = plt.figure()
    a1 = fig.add_axes([0, 0, 1, 1])
    a1.plot((raw[channel_number - 1][0].T))
    a1.plot((raw_f[channel_number - 1][0].T))
    a1.legend(["raw", "band-pass filtered"])
    plt.title("Subject: " + str(subject_id) + ", channel: " + str(channel_number))
    plt.ylabel("potential [µV]")
    plt.xlabel("sample")
    a1.set_xlim(0, 25)
    a1.set_ylim(0.0199, 0.01993)
    ccs_eeg_utils.save_plot("04zoom_raw", bids_path)
    plt.close()  # don't display figure

    # show filtered -> high frequnecies are filtered
    fig = plt.figure()
    a1 = fig.add_axes([0, 0, 1, 1])
    a1.plot((raw[channel_number - 1][0].T))
    a1.plot((raw_f[channel_number - 1][0].T))
    a1.legend(["raw", "band-pass filtered"])
    plt.title("Subject: " + str(subject_id) + ", channel: " + str(channel_number))
    plt.ylabel("potential [µV]")
    plt.xlabel("sample")
    a1.set_xlim(0, 25)
    a1.set_ylim(-0.00001, 0.00002)
    ccs_eeg_utils.save_plot("05zoom_filtered", bids_path)
    plt.close()  # don't display figure

In [7]:
def cleaning(raw_f, subject):
    """
    clean and plot
    """
    # read bad annotations
    filename = str("./../results/" + str(bids_path)[14:-37]) + str(
        "badannotations" + ".csv"
    )

    # read annotations
    if subject == 1 or subject == 2 or subject == 3:
        annotations = mne.read_annotations(filename)

        # append bad annotations
        raw_f.annotations.append(
            annotations.onset, annotations.duration, annotations.description
        )  # duplicates

    # clean manually
    #%matplotlib qt
    duration = [5, 5, 10]  # duration for plot
    start = [209, 231, 95]  # start time for plot
    raw_f.plot(
        n_channels=len(raw_f.ch_names),
        duration=duration[subject - 1],
        start=start[subject - 1],
    )  # interactive plot
    if closeInteractiveCleaningPlot:
        ccs_eeg_utils.save_plot("050cleaning_data", bids_path)

    # To modify annotations, use plot and run next two functions of getting and saving annotations
    # get bad annotations
    """bad_ix = [
        i for i, a in enumerate(raw_f.annotations) if a["description"] == "BAD_"
    ]
    
    # save bad annotations
    raw_f.annotations[bad_ix].save(
        str('./../results/' + str(bids_path)[14:-37])+str("badannotations"
            + ".csv"
        )
    )"""

    # remove bad channels
    raw_f.info["bads"] = []

    # intepolate bad channels
    # raw_f.interpolate_bads()  # no bad channel found

    # save cleaned data
    raw_f.save(
        str("./../results/" + str(bids_path)[14:-37]) + str("after_cleaning_raw.fif"),
        overwrite=True,
    )

    # load cleaned data
    # raw_cleaned = mne.io.read_raw_fif(str('./../results/' + str(bids_path)[14:-37])+str("after_cleaning_raw.fif"))

    # visualize effect of cleaning on ERP
    evts, evts_dict = mne.events_from_annotations(raw)
    wanted_keys = [e for e in evts_dict.keys() if "stimulus" in e]
    evts_dict_stim = dict((k, evts_dict[k]) for k in wanted_keys if k in evts_dict)

    # get raw epochs without rejections
    epochs = mne.Epochs(
        raw_f,
        evts,
        evts_dict_stim,
        tmin=epoch_tmin,
        tmax=epoch_tmax,
        reject_by_annotation=False,
        baseline=baseline,
    )

    # get epochs with manual rejection
    epochs_manual = mne.Epochs(
        raw_f,
        evts,
        evts_dict_stim,
        tmin=epoch_tmin,
        tmax=epoch_tmax,
        reject_by_annotation=True,
        baseline=baseline,
    )

    # get epochs with threshold rejection
    reject_criteria = dict(
        eeg=rejection_threshold,
        eog=rejection_threshold,
    )
    epochs_thresh = mne.Epochs(
        raw_f,
        evts,
        evts_dict_stim,
        tmin=epoch_tmin,
        tmax=epoch_tmax,
        reject=reject_criteria,
        reject_by_annotation=False,
        baseline=baseline,
    )

    # get epochs with Manual + theshold rejection
    epochs__manual_and_thresh = mne.Epochs(
        raw_f,
        evts,
        evts_dict_stim,
        tmin=epoch_tmin,
        tmax=epoch_tmax,
        reject=reject_criteria,
        reject_by_annotation=True,
        baseline=baseline,
    )

    # plot comparison
    mne.viz.plot_compare_evokeds(
        {
            "raw": epochs.average(),
            "manual": epochs_manual.average(),
            "thresh": epochs_thresh.average(),
            "both": epochs__manual_and_thresh.average(),
        },
        picks=picked_channel,
    )

    # save amd close plot
    ccs_eeg_utils.save_plot("05cleaning_compare_evoked", bids_path)

    return raw_f

In [8]:
def add_events(raw):
    """
    add events
    """
    print("adding events ...")
    evts, evts_dict = mne.events_from_annotations(raw_f)

    # get all keys which contain "stimulus"
    wanted_keys = [
        e for e in evts_dict.keys() if "stimulus" in e
    ]  # subset the large event-dictionairy

    evts_dict_stim = dict((k, evts_dict[k]) for k in wanted_keys if k in evts_dict)

    return evts, evts_dict_stim

In [9]:
def ICA(raw_f_f, subject):
    """
    ICA
    """
    print("running ICA ...")
    # apply a high pass filter of 1.5 Hz
    raw_f_f = raw_f.copy()
    raw_f_f.filter(l_freq_ica, h_freq_ica, fir_design=fir_design_ica)

    # Generate an ICA object with fixed random state to make it deterministic, thus reproducible
    ica = mne.preprocessing.ICA(
        n_components=0.9999999, random_state=93, method=ica_method, max_iter=200
    )

    # fit the ICA on the data, where the 1.5Hz high pass filter was applied.
    # Also omit bad segments(manual cleaning annotations) from the data before fitting.
    ica.fit(raw_f_f, reject_by_annotation=True, verbose=True)

    # plot components
    ica.plot_components(picks=range(ica.n_components_ - 1))
    ccs_eeg_utils.save_plot("06ICA_components", bids_path)

    # plot properties
    # create dir to save plots if not existent
    if not os.path.exists(
        str("./../results/" + str(bids_path)[14:-37]) + str("07ICA_properties/")
    ):
        os.makedirs(
            str("./../results/" + str(bids_path)[14:-37]) + str("07ICA_properties/")
        )
    for component in range(ica.n_components_):
        ica.plot_properties(
            raw_f, picks=component, psd_args={"fmax": 35.0}, reject=None
        )
        # save plot
        plt.savefig(
            str("./../results/" + str(bids_path)[14:-37])
            + str("07ICA_properties/component" + str(component) + ".png"),
            bbox_inches="tight",
        )

        # don't display figure
        plt.close()

    # Get artefacts of each subject
    if subject == 1:
        heartbeat_artefacts = heart_sub1
        blink_artefacts = blink_sub1
        eye_artefacts = eye_sub1
        muscle_artefacts = muscle_sub1  # often non-stationary activity, small ERPs, "square-root spectrum", one electrode?, Yaw, below/behind ear, lowwe back of the head,
        noisy_electrodes = noisy_electrode_sub1  # single active electrode, small ERP, strong 50/60hz line noise
        other_artefacts = other_sub1
        unsure_artefacts = unsure_sub1
        # brain components ideally show strong and clear ERPs, activity stationary over whole experiment, powerspectrum including alpha/beta peak at ~10hz,

    elif subject == 2:
        heartbeat_artefacts = heart_sub2
        blink_artefacts = blink_sub2
        eye_artefacts = eye_sub2
        muscle_artefacts = muscle_sub2
        noisy_electrodes = noisy_electrode_sub2
        other_artefacts = other_sub2
        unsure_artefacts = unsure_sub2

    elif subject == 3:
        heartbeat_artefacts = heart_sub3
        blink_artefacts = blink_sub3
        eye_artefacts = eye_sub3
        muscle_artefacts = muscle_sub3
        noisy_electrodes = noisy_electrode_sub3
        other_artefacts = other_sub3
        unsure_artefacts = unsure_sub3

    # exclude artefacts
    exclude = []
    exclude.extend(heartbeat_artefacts)
    exclude.extend(blink_artefacts)
    exclude.extend(eye_artefacts)
    exclude.extend(muscle_artefacts)
    exclude.extend(other_artefacts)
    exclude.extend(unsure_artefacts)
    print("excluded components:", exclude)
    ica.exclude = exclude

    # add events
    evts, evts_dict_stim = add_events(raw_f)

    # plot before ICA
    epochs = mne.Epochs(
        raw_f,
        evts,
        evts_dict_stim,
        tmin=epoch_tmin,
        tmax=epoch_tmax,
        baseline=baseline,
        reject_by_annotation=True,
    )
    epochs.average().plot(picks=picked_channel)
    ccs_eeg_utils.save_plot("08" + str(picked_channel) + "_before_ICA", bids_path)

    # before / after overlay
    reconst_raw_f = raw_f.copy()
    ica.apply(reconst_raw_f, exclude=exclude)

    # plot after ICA
    epochs = mne.Epochs(
        reconst_raw_f,
        evts,
        evts_dict_stim,
        tmin=epoch_tmin,
        tmax=epoch_tmax,
        baseline=baseline,
    )
    epochs.average(picks=picked_channel).plot()  # 13 for Pz?
    ccs_eeg_utils.save_plot("09" + str(picked_channel) + "_after_ICA", bids_path)
    print(epochs.info)

    # raw_f.plot()
    # reconst_raw_f.plot()
    ica.plot_overlay(raw_f, exclude=exclude, picks="eeg")
    ccs_eeg_utils.save_plot("10before_after_overlay", bids_path)

    # save ICA
    ica.save(str(str(bids_path)[:-4] + "_chrei-ica.fif"))

    return reconst_raw_f

In [10]:
def epoching():
    # Choosing Pz for P3 because this is the site most widely used in the literature and
    # recommended by Kappenman et al.
    # Cut the raw_f data to one channel using `raw_f.pick_channels(["Pz"])` -

    evts, evts_dict_stim = add_events(raw_f)

    raw_f.pick_channels([picked_channel])

    # Epoch the data
    epochs = mne.Epochs(
        raw_f, evts, evts_dict_stim, tmin=epoch_tmin, tmax=epoch_tmax, baseline=baseline
    )  # tmin and tmax chosen as recommended by Kappenman et al.

    # Plot all trials
    plt.plot(np.squeeze(epochs.get_data()[:, 0, :].T))
    ccs_eeg_utils.save_plot("12epochs", bids_path)

    # delay/time shift
    # https://mne.tools/stable/generated/mne.Epochs.html?highlight=delay%20shift
    # todo
    # epochs.load_data()
    # epochs.shift_time(0.026, relative=True)  # for LCD delay: as in Kappenman et al.
    # plt.plot(np.squeeze(epochs.get_data()[:, 0, :].T))
    # ccs_eeg_utils.save_plot("12epochs_after_shift_time", bids_path)

    return epochs, evts, evts_dict_stim

In [11]:
def get_peak_value(data, timestamp):
    """
    get peak values from timestamp
    """
    peak_time_index = data.time_as_index(timestamp)
    data_frame = data.to_data_frame(picks=picked_channel)
    peak_value = data_frame.at[peak_time_index[0], picked_channel]

    return peak_value

In [12]:
def single_subject_erp_analysis(epochs, evts, evts_dict_stim):
    # indexing and averaging epochs
    target = epochs[["stimulus:{}{}".format(k, k) for k in [1, 2, 3, 4, 5]]].average()
    target_evoked_list.append(target)
    
    distractor = epochs[
        [
            "stimulus:{}{}".format(k, j)
            for k in [1, 2, 3, 4, 5]
            for j in [1, 2, 3, 4, 5]
            if k != j
        ]
    ].average()
    distractor_evoked_list.append(distractor)

    # plotting
    mne.viz.plot_compare_evokeds(
        dict(target=target, distractor=distractor), legend=True, show_sensors="upper right"
    )
    ccs_eeg_utils.save_plot("13epochs_average", bids_path)

    """
    get peaks
    """
    # get peak times target
    _, peak_time_target = target.get_peak(mode="pos")  # get peak time
    print("peak_time_target:", peak_time_target)
    peak_time_target_list.append(peak_time_target)

    # get peak value target
    peak_value_target = get_peak_value(target, peak_time_target)
    print("peak_value_target:", peak_value_target)
    peak_value_target_list.append(peak_value_target)

    # get peak times distractor
    _, peak_time_distractor = distractor.get_peak(mode="pos")
    print("peak_time_distractor:", peak_time_distractor)
    peak_time_distractor_list.append(peak_time_distractor)

    # get peak value distractor
    peak_value_distractor = get_peak_value(distractor, peak_time_distractor)
    print("peak_value_distractor:", peak_value_distractor)
    peak_value_distractor_list.append(peak_value_distractor)

    return (
        peak_time_target_list,
        peak_value_target_list,
        peak_time_distractor_list,
        peak_value_distractor_list,
        target_evoked_list,
        distractor_evoked_list,
    )

In [13]:
def create_results_df(
    peak_time_target_list,
    peak_value_target_list,
    peak_time_distractor_list,
    peak_value_distractor_list,
):
    # create subject ID
    sub_id = list(range(len(peak_value_distractor_list)))
    sub_id = np.add(sub_id,1) 
    
    # compute diff time
    diff_time = np.subtract(peak_time_target_list,peak_time_distractor_list)
    
    # compute diff value
    diff_value = np.subtract(peak_value_target_list,peak_value_distractor_list)
    
    #create df
    results_dict = dict(
        subject_id = sub_id,
        peak_time_target=peak_time_target_list,
        peak_time_distractor=peak_time_distractor_list,
        peak_amplitude_target=peak_value_target_list,
        peak_amplitude_distractor=peak_value_distractor_list,
        diff_time=diff_time,
        diff_amplitude=diff_value,
    )
    results_df = pd.DataFrame.from_dict(results_dict)

    return results_df

In [14]:
def erp_differences(
    peak_target_list,
    peak_distractor_list,
    title="title",
    xlabel="xlabel",
    ylabel="# subjects",
):
    peak_difference_list = np.subtract(peak_target_list, peak_distractor_list)
    #%matplotlib inline
    _ = plt.hist(
        peak_difference_list, bins="auto"
    )  # arguments are passed to np.histogram
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    #plt.show()
    
    # save fig
    plt.savefig(
        "../intersubject_results/"+str(title)+".png",
        bbox_inches="tight",
    )
    # plt.show()
    plt.close()  # don't display figure    

    average_difference = np.average(peak_difference_list)
    meadian_difference = np.median(peak_difference_list)
    std_dev_difference = np.std(peak_difference_list)

    # print(average_difference, meadian_difference, std_dev_difference)
    return average_difference, meadian_difference, std_dev_difference

In [23]:
def permutation_test(peak_target,peak_distractor,title):
    # create one data array with peak values and labels
    peak_data = np.array([peak_target, peak_distractor])
    print("peak_data", peak_data)

    # perform permutation test
    t_obs, p_values, H0 = mne.stats.permutation_t_test(peak_data.T, verbose=True)
    print(title,"t_obs", t_obs)
    print(title,"p_values:", p_values)
    print(title,"H0:", H0)


    # plot results
    fig, ax = plt.subplots()
    ax.hist(H0, bins=50, density=True)
    ax.axvline(t_obs[0], ls="--", color="r")
    ax.axvline(t_obs[1], ls="--", color="b")

    # ax.text(0.7, 260, score_label, fontsize=12)
    ax.set_xlabel(f"Accuracy score. p-value={p_values[1]:.3f}")
    _ = ax.set_ylabel("Probability")

    # save fig
    plt.savefig(
        "../intersubject_results/"+str(title)+".png",
        bbox_inches="tight",
    )
    plt.close()  # don't display figure"""

In [16]:
def plot_time_of_peak_distractor_vs_target(results_df):
    # scatterplot time_of_peak_distractor_vs_target

    # get extreme values for axis
    max_target = results_df['peak_time_target'].max()
    max_distractor = results_df['peak_time_distractor'].max()
    max_both = max(max_target,max_distractor)
    min_target = results_df['peak_time_target'].min()
    min_distractor = results_df['peak_time_distractor'].min()
    min_both = min(min_target,min_distractor)

    # plot
    g=sns.jointplot(
        data=results_df,
        x="peak_time_target",
        y="peak_time_distractor",
        ylim = (min_both-0.1,max_both+0.1),
        xlim=(min_both-0.1,max_both+0.1),
    )
    g.plot_joint(sns.kdeplot, color="r", zorder=0, levels=1)
    # save fig
    plt.savefig(
        "../intersubject_results/time_of_peak_distractor_vs_target.png",
        bbox_inches="tight",
    )
    plt.close()  # don't display figure


In [17]:
def plot_peak_amplitude_distractor_vs_target(results_df):
    # scatterplot peak_amplitude_distractor_vs_target

    # get extreme values for axis
    max_target = results_df['peak_amplitude_target'].max()
    max_distractor = results_df['peak_amplitude_target'].max()
    max_both = max(max_target,max_distractor)
    min_target = results_df['peak_amplitude_target'].min()
    min_distractor = results_df['peak_amplitude_target'].min()
    min_both = min(min_target,min_distractor)


    g=sns.jointplot(
        data=results_df,
        x="peak_amplitude_target",
        y="peak_amplitude_distractor",
        ylim = (min_both-1,max_both+1),
        xlim=(min_both-1,max_both+1),
    )
    g.plot_joint(sns.kdeplot, color="r", zorder=0, levels=1)

    # save fig
    plt.savefig(
        "../intersubject_results/peak_amplitude_distractor_vs_target.png",
        bbox_inches="tight",
    )
    plt.close()  # don't display figure

In [18]:
def inter_subject_analysis(
    results_df,
    target_evoked_list,
    distractor_evoked_list,
):
    """
    permutation t tests:
    """
    # time delay of peak
    permutation_test(
        results_df["peak_time_target"],
        results_df["peak_time_distractor"],
        title="permutation_time",
    )
    
    # difference in peak potential
    permutation_test(
        np.array(results_df["peak_amplitude_target"]),
        np.array(results_df["peak_amplitude_distractor"]),
        title="permutation_distractor",
    )


    """
    pemutation cluster test: TODO didn't get it
    """
    from mne.stats import permutation_cluster_test

    # target_evoked_array = np.array(target_evoked_list)
    # distractor_evoked_array = np.array(distractor_evoked_list)
    # target_evoked_array = np.array(target_evoked_list)

    T_obsC, clustersC, cluster_p_valuesC, H0 = permutation_cluster_test(
        [
            np.array(results_df["peak_time_target"]),
            np.array(results_df["peak_time_distractor"]),
        ],
        # [target_evoked_array,distractor_evoked_array],
        n_permutations=100,
        threshold=None,
    )
    print(
        "T_obsC:",
        T_obsC,
        "\nclustersC:",
        clustersC,
        "\ncluster_p_valuesC:",
        cluster_p_valuesC,
        "\nHO",
        H0,
    )

    """
    combine evoked
    """
    # combine to averaged evoked
    target_evoked_averaged = mne.combine_evoked(target_evoked_list, weights="equal")
    distractor_evoked_averaged = mne.combine_evoked(
        distractor_evoked_list, weights="equal"
    )
    # calculate difference wave
    difference_averaged_evoked = mne.combine_evoked(
        [target_evoked_averaged, distractor_evoked_averaged], weights=[1, -1]
    )

    # plot compare evoked
    #%matplotlib inline
    mne.viz.plot_compare_evokeds(
        dict(
            target=target_evoked_averaged,
            distractor=distractor_evoked_averaged,
            difference=difference_averaged_evoked,
        ),
        legend="upper left",
        show_sensors="upper right",
    )
    # save fig
    plt.savefig(
        "../intersubject_results/inter_subject_target_distractor_difference.png",
        bbox_inches="tight",
    )
    plt.close()  # don't display figure

    """
    Descriptive Statistics
    """
    # save results as CSV
    # save all_subjects_time_amplitude results to file
    results_df.to_csv(
        index=False,
        path_or_buf="./../intersubject_results/all_subjects_time_amplitude.csv",
    )

    # compute and save statistics to file. drop statistics over subject id
    results_df.drop(["subject_id"], axis=1).describe().to_csv(
        index=True, path_or_buf="./../intersubject_results/statistics.csv"
    )

    # compare peak_time target vs distractor in boxplot
    results_df.boxplot(column=["peak_time_target", "peak_time_distractor", "diff_time"])
    plt.savefig(
        "../intersubject_results/time_boxplot.png",
        bbox_inches="tight",
    )
    plt.close()  # don't display figure

    # compare peak_amplitude target vs distractor in boxplot
    results_df.boxplot(
        column=["peak_amplitude_target", "peak_amplitude_distractor", "diff_amplitude"]
    )
    plt.savefig(
        "../intersubject_results/amplitude_boxplot.png",
        bbox_inches="tight",
    )
    plt.close()  # don't display figure

    # scatter plots
    plot_time_of_peak_distractor_vs_target(results_df)
    plot_peak_amplitude_distractor_vs_target(results_df)

In [19]:
def source_space_localization(raw):
    """
    TODO check if raw_f needed, but here raw because raw_f has only Pz channel
    TODO 
    TODO individually or whole goup average?
    """
    # set eeg reference because EEG average reference is mandatory for inverse modeling
    raw.set_eeg_reference(projection=True)

    # get epochs
    epochs = mne.Epochs(
        raw, evts, evts_dict_stim, tmin=epoch_tmin, tmax=epoch_tmax, baseline=baseline
    )

    # get evoked
    target = epochs[["stimulus:{}{}".format(k, k) for k in [1, 2, 3, 4, 5]]].average()
    distractor = epochs[
        [
            "stimulus:{}{}".format(k, j)
            for k in [1, 2, 3, 4, 5]
            for j in [1, 2, 3, 4, 5]
            if k != j
        ]
    ].average()

    """ TODO also for distractors?"""
    evoked = target
    
    # plot_joint to check
    #%matplotlib inline
    evoked.plot_joint()
    plt.savefig(
        "../intersubject_results/butterfly_before_source_space.png",
        bbox_inches="tight",
    )
    plt.close()  # don't display figure  

    # get average src and bem as shown in the following tutorial:
    # https://mne.tools/stable/auto_tutorials/source-modeling/plot_eeg_no_mri.html
    fs_dir = fetch_fsaverage(verbose=True)
    subjects_dir = op.dirname(fs_dir)
    subject = "fsaverage"
    trans = "fsaverage"  # MNE has a built-in fsaverage transformation
    src = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
    bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")

    # plot to check alignment
    """fig = plot_alignment( # TODO back on
        evoked.info,
        trans='fsaverage',
        show_axes=True,
        surfaces="head-dense",
        subject=subject,
        subjects_dir=subjects_dir,
    )"""
    """ #TODO unable to save
    plt.savefig(
        "../intersubject_results/alignment.png",
        bbox_inches="tight",
    )
    plt.close()  # don't display figure  
    """

    # Calculate a forward solution for a subject
    fwd = mne.make_forward_solution(
        evoked.info, trans=None, src=src, bem=bem, verbose=True
    )

    # Estimate noise covariance matrix from epochs.
    cov = mne.compute_covariance(epochs, tmax=0.0)
    %matplotlib inline
    cov.plot(epochs.info)
    plt.savefig( # save fig
        "../intersubject_results/noise_covariance.png",
        bbox_inches="tight",
    )
    #plt.close()  # don't display figure 


    # Assemble inverse operator
    #inv = mne.minimum_norm.make_inverse_operator(evoked.info, fwd, cov, verbose=True)

    # Apply inverse operator to evoked data.
    #stc = mne.minimum_norm.apply_inverse(evoked, inv)

    # plot source space
    #brain = stc.plot(subjects_dir=subjects_dir, initial_time=0.1)

In [20]:
"""
pipeline
"""
# path where to save the datasets.
bids_root = "../local/bids"
last_subject

# time of peak
target_evoked_list = []
distractor_evoked_list = []
peak_time_target_list = []
peak_time_distractor_list = []
peak_value_target_list = []
peak_value_distractor_list = []

for subject in range(first_subject, last_subject + 1):
    print("subject =", subject)
    subject_id = f"{subject:03}"

    bids_path = BIDSPath(
        subject=subject_id,
        task="P3",
        session="P3",
        datatype="eeg",
        suffix="eeg",
        root=bids_root,
    )

    # read the file
    raw = read_raw_bids(bids_path)

    # fix the annotations readin
    ccs_eeg_utils.read_annotations_core(bids_path, raw)

    # load data
    raw.load_data()

    # downsample frequency
    downsample()

    rereference()

    # apply montage
    apply_montage()
    
    # band pass filtering between 0.4 and 54 hz
    raw_f = band_pass_filtering(subject_id)

    # plot each channel
    plot_each_channel(subject_id) # TODO turn back on

    # remove large muscle artifacts, extreme voltage offsets, or break periods
    if subject == 1 or subject ==2 or subject ==3:
        cleaning(raw_f, subject)

        # ICA
        raw_f = ICA(raw_f, subject)

    epochs, evts, evts_dict_stim = epoching()
    
    (
        peak_time_target_list,
        peak_value_target_list,
        peak_time_distractor_list,
        peak_value_distractor_list,
        target_evoked_list,
        distractor_evoked_list,
    ) = single_subject_erp_analysis(epochs, evts, evts_dict_stim)

# create results df
results_df = create_results_df(
    peak_time_target_list,
    peak_value_target_list,
    peak_time_distractor_list,
    peak_value_distractor_list,
)

# inter-subject peak analysis
inter_subject_analysis(
    results_df,
    target_evoked_list,
    distractor_evoked_list,
)


# source space localization
source_space_localization(raw)

subject = 1
Reading ../local/bids/sub-001/ses-P3/eeg/sub-001_ses-P3_task-P3_eeg.fdt
Reading events from ../local/bids/sub-001/ses-P3/eeg/sub-001_ses-P3_task-P3_events.tsv.
Reading channel info from ../local/bids/sub-001/ses-P3/eeg/sub-001_ses-P3_task-P3_channels.tsv.
Reading 0 ... 478207  =      0.000 ...   466.999 secs...


<ipython-input-20-2765d89c3d62>:30: RuntimeWarning: Data file name in EEG.data (sub-001_task-P3_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-001_ses-P3_task-P3_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-20-2765d89c3d62>:30: RuntimeWarning: Did not find any coordsystem.json associated with sub-001_ses-P3_task-P3.

The search_str was "../local/bids/sub-001/**/sub-001_ses-P3*coordsystem.json"
  raw = read_raw_bids(bids_path)


before resampling: <Info | 9 non-empty values
 bads: []
 ch_names: FP1, F3, F7, FC3, C3, C5, P3, P7, P9, PO7, PO3, O1, Oz, Pz, CPz, ...
 chs: 30 EEG, 3 EOG
 custom_ref_applied: False
 highpass: 0.0 Hz
 line_freq: 60
 lowpass: 512.0 Hz
 meas_date: unspecified
 nchan: 33
 projs: []
 sfreq: 1024.0 Hz
 subject_info: 4 items (dict)
>
after resampling: <Info | 9 non-empty values
 bads: []
 ch_names: FP1, F3, F7, FC3, C3, C5, P3, P7, P9, PO7, PO3, O1, Oz, Pz, CPz, ...
 chs: 30 EEG, 3 EOG
 custom_ref_applied: False
 highpass: 0.0 Hz
 line_freq: 60
 lowpass: 128.0 Hz
 meas_date: unspecified
 nchan: 33
 projs: []
 sfreq: 256.0 Hz
 subject_info: 4 items (dict)
>
EEG channel type selected for re-referencing
Applying a custom EEG reference.
<Info | 11 non-empty values
 bads: []
 ch_names: FP1, F3, F7, FC3, C3, C5, P3, P7, P9, PO7, PO3, O1, Oz, Pz, CPz, ...
 chs: 30 EEG, 3 EOG
 custom_ref_applied: True
 dig: 33 items (3 Cardinal, 30 EEG)
 highpass: 0.0 Hz
 line_freq: 60
 lowpass: 128.0 Hz
 meas_date

    Rejecting  epoch based on EOG : ['HEOG_left', 'HEOG_right', 'VEOG_lower']
    Rejecting  epoch based on EOG : ['HEOG_left', 'HEOG_right', 'VEOG_lower']
    Rejecting  epoch based on EEG : ['FP2']
    Rejecting  epoch based on EOG : ['HEOG_left', 'HEOG_right', 'VEOG_lower']
    Rejecting  epoch based on EOG : ['HEOG_right']
    Rejecting  epoch based on EEG : ['FP1', 'FP2', 'F8']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EOG : ['HEOG_right']
    Rejecting  epoch based on EEG : ['F8']
    Rejecting  epoch based on EEG : ['F8']
    

<ipython-input-20-2765d89c3d62>:30: RuntimeWarning: Did not find any coordsystem.json associated with sub-002_ses-P3_task-P3.

The search_str was "../local/bids/sub-002/**/sub-002_ses-P3*coordsystem.json"
  raw = read_raw_bids(bids_path)


...

before resampling: <Info | 9 non-empty values
 bads: []
 ch_names: FP1, F3, F7, FC3, C3, C5, P3, P7, P9, PO7, PO3, O1, Oz, Pz, CPz, ...
 chs: 30 EEG, 3 EOG
 custom_ref_applied: False
 highpass: 0.0 Hz
 line_freq: 60
 lowpass: 512.0 Hz
 meas_date: unspecified
 nchan: 33
 projs: []
 sfreq: 1024.0 Hz
 subject_info: 4 items (dict)
>
after resampling: <Info | 9 non-empty values
 bads: []
 ch_names: FP1, F3, F7, FC3, C3, C5, P3, P7, P9, PO7, PO3, O1, Oz, Pz, CPz, ...
 chs: 30 EEG, 3 EOG
 custom_ref_applied: False
 highpass: 0.0 Hz
 line_freq: 60
 lowpass: 128.0 Hz
 meas_date: unspecified
 nchan: 33
 projs: []
 sfreq: 256.0 Hz
 subject_info: 4 items (dict)
>
EEG channel type selected for re-referencing
Applying a custom EEG reference.
<Info | 11 non-empty values
 bads: []
 ch_names: FP1, F3, F7, FC3, C3, C5, P3, P7, P9, PO7, PO3, O1, Oz, Pz, CPz, ...
 chs: 30 EEG, 3 EOG
 custom_ref_applied: True
 dig: 33 items (3 Cardinal, 30 EEG)
 highpass: 0.0 Hz
 line_freq: 60
 lowpass: 128.0 Hz
 meas_date

<ipython-input-20-2765d89c3d62>:30: RuntimeWarning: Data file name in EEG.data (sub-040_task-P3_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-040_ses-P3_task-P3_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-20-2765d89c3d62>:30: RuntimeWarning: Did not find any coordsystem.json associated with sub-040_ses-P3_task-P3.

The search_str was "../local/bids/sub-040/**/sub-040_ses-P3*coordsystem.json"
  raw = read_raw_bids(bids_path)


before resampling: <Info | 9 non-empty values
 bads: []
 ch_names: FP1, F3, F7, FC3, C3, C5, P3, P7, P9, PO7, PO3, O1, Oz, Pz, CPz, ...
 chs: 30 EEG, 3 EOG
 custom_ref_applied: False
 highpass: 0.0 Hz
 line_freq: 60
 lowpass: 512.0 Hz
 meas_date: unspecified
 nchan: 33
 projs: []
 sfreq: 1024.0 Hz
 subject_info: 4 items (dict)
>
after resampling: <Info | 9 non-empty values
 bads: []
 ch_names: FP1, F3, F7, FC3, C3, C5, P3, P7, P9, PO7, PO3, O1, Oz, Pz, CPz, ...
 chs: 30 EEG, 3 EOG
 custom_ref_applied: False
 highpass: 0.0 Hz
 line_freq: 60
 lowpass: 128.0 Hz
 meas_date: unspecified
 nchan: 33
 projs: []
 sfreq: 256.0 Hz
 subject_info: 4 items (dict)
>
EEG channel type selected for re-referencing
Applying a custom EEG reference.
<Info | 11 non-empty values
 bads: []
 ch_names: FP1, F3, F7, FC3, C3, C5, P3, P7, P9, PO7, PO3, O1, Oz, Pz, CPz, ...
 chs: 30 EEG, 3 EOG
 custom_ref_applied: True
 dig: 33 items (3 Cardinal, 30 EEG)
 highpass: 0.0 Hz
 line_freq: 60
 lowpass: 128.0 Hz
 meas_date

<ipython-input-18-90829bff7c37>:33: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  T_obsC, clustersC, cluster_p_valuesC, H0 = permutation_cluster_test(
<ipython-input-18-90829bff7c37>:33: DeprecationWarning: The default for "out_type" will change from "mask" to "indices" in version 0.22. To avoid this warning, explicitly set "out_type" to one of its string values.
  T_obsC, clustersC, cluster_p_valuesC, H0 = permutation_cluster_test(
<ipython-input-18-90829bff7c37>:33: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_obsC, clustersC, cluster_p_valuesC, H0 = permutation_cluster_test(


[] 
HO []
